# Latihan 2 - Algoritma Shor's
## Latar belakang sejarah

Didalam komputasi, kita sering mengukur peforma dari algoritma dengan bagaimana algoritma tersebut berkembang dengan masukkan yang diberikan. Sebagai contoh, penjumlahan memiliki algoritma yang berkembang secara linear dengan ukuran angka yang kita tambahkan. Ada beberapa masalah didalam komputasi yang dimana memiliki algoritma yang kita harus berkembang secara _eksponensial_ dengan ukuran masukkan yang diberikan, dan dimana masukkan yang memiliki ukuran yang terlalu besar untuk diselesaikan oleh komputer diseluruh dunia. Kita sangat yakin,sebagian dari keamanan internet bergantung pada masalah komputasi yang tidak dapat di pecahkan.

PAda tahun 1994, Peter Shor menunjukan bahwa sangatlah mungkin untuk memfaktorisasi angka menjadi angka prima secara efektif di komputer kuantum.[1] Yang merupakan berita besar, sebagai algoritma klasikal yang kita ketahui yang dimana algoritma tersebut dapat berkembang secara eksponensial. Dan sebagai kenyataannya, [Enkripsi RSA](https://id.wikipedia.org/wiki/RSA) bergantung pada pemfaktoran angka yang besar sehingga menjadi tidak mungkin untuk dipecahkan. Untuk memfaktorisasi angka integer yang sangat besar pada komputer klasik diperlukan jutaan qubit dan gerbang, dan sirkuit ini terlalu besar untuk dieksekusi oleh kuantum komputer pada saat ini.

Jadi bagaimana Lieven M.K. Vandersypen, Matthias Steffen, Gregory Breyta, Costantino S. Yannoni, Mark H. Sherwood and Isaac L. Chuang berhasil memfaktorkan 15 di kuantum komputer pada tahun 2001?! [2]

Kesulitan dalam membuat sirkuit untuk algoritma Shor’s adalah membuat sirkuit yang dapat menghitung $ay \bmod N$ secara terkontrol. Ketika kita tahu bagaimana membuat sirkuit ini dengan gerbang secara polinomial, yang masih terlalu besar untuk komputer saat ini. Beruntung, jika kita telah mengetahui sebagian dari informasi dari pada awal masalah, kemudia kita dapat 'mencurangi' dan membuat sirkuit yang lebih efisien.

Untuk menjalakan sirkuit di perakgkat keras yang ada pada saat itu, penulis jurnal diatas mendapatkan sirkuit yang simpel yang dapat mengeksekusi $7y \bmod 15$. Yang dimana membuat sirkuit tersebut cukup kecil untuk dijalankan di perangkat keras mereka. Pada akhir dari latihan ini, anda akan membuat sirkuit untuk $35y \bmod N$ yang dapat digunakan di algoritma Shor’s dan dapat dijalankan di `ibmq_santiago`.

Jika anda ingin megrti apa yang terjadi didalam latihan ini, anda dapat membaca [buku cetak Qiskit halaman algoritma Shor’s](https://qiskit.org/textbook/ch-algorithms/shor.html), jika ini terlalu susah bagi anda, anda dapat menyelesaikan latihan ini tanpa membaca.

### Referensi
1. Shor, Peter W. "Algorithms for quantum computation: discrete logarithms and factoring." Proceedings 35th annual symposium on foundations of computer science. Ieee, 1994.
2. Vandersypen, Lieven MK, et al. "Experimental realization of Shor's quantum factoring algorithm using nuclear magnetic resonance." Nature 414.6866 (2001): 883-887.

## tl;dr: Algoritma Shor's

Ada satu algoritma disebut [_quantum phase estimation_](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html) yang dimana memberitahukan gelombang yang di berikan dari gerbang untuk setiap keadaan kuantum. Sebagai contoh, masukan untuk algoritma estimasi gelombang dapat menjadi keadaan kuantum $|1\rangle$ dan gerbang $Z$. Jika gerbang $Z$ bereaksi pada keadaan $|1\rangle$, kita mendapatkan keadaan yang sama dengan pertambahaan gelombang global sebesar $\pi$:

$$
Z|1\rangle = -|1\rangle = e^{i\pi} |1\rangle
$$

dan *quantum phase estimation* dapat menghasilkan untuk kita. Anda dapat melihat contoh lain [disini](https://qiskit.org/textbook/ch-algorithms/quantum-phase-estimation.html#2.-Example:-T-gate-).

Shor menunjukan bhawa kita melakukan estimasi gelombang dengan gerbang, $U$, yang memiliki sifat $U|y\rangle = |a y\bmod N\rangle$, kita dapat mendapatkan beberapa informasi faktor dari $N$.

2## Masalah

Didalam latihan ini, kita akan memfaktorisasi 25 dengan menggunakan estimasi gelombang dengan sirkuit yang mengimplentasikan $13y \bmod 35$. Didalam latihan ini membuat sirkuit dan cukup kecil untuk dijalankan di `ibmq_santiago`! Ini berupakan tugas yang mudah, jadi pertama sekali kita membuat contekan.

Detil dari algoritma Shor’s adalah sirkuit kita haruslah berkerja pada keadaan yang kita capai dengan mengaplikasikan $U$ untuk memulai keadaan $|1\rangle$. Contoh kita dapat menggunakan sirkuit apapun yang memiliki sifat: 

$$
\begin{aligned}
U|1\rangle &= |13\rangle \\
UU|1\rangle &= |29\rangle \\
UUU|1\rangle &= |27\rangle \\
UUUU|1\rangle &= |1\rangle \\
\end{aligned}
$$

Jadi bagaimana kita membuat menjadi lebih mudah untuk kita ? Sejak kita hanya memerlukan untuk mentransformasi 4 keadaan berbeda secara benar. Untuk latihan ini, kita akan memilih untuk memetakan 2 qubit keadaan komputasional basis ke angka seperti:

$$
\begin{aligned}
|1\rangle &\rightarrow |00\rangle \\
|13\rangle &\rightarrow |01\rangle \\
|29\rangle &\rightarrow |10\rangle \\
|27\rangle &\rightarrow |11\rangle \\
\end{aligned}
$$

Kenapa ini “menyontek”? Karena, untuk menggunakan optimisasi ini, kita perlu tahu semua keadaan $U$ yang akan terefek, yang dimana kita harus menghitung $ay \bmod N$ sampai kita mendapatkan 1 kembali, dan artinya kita mengetahui periode dari $a^x \bmod N$ dan artinya kita mendapatkan faktor dari $N$. Semua optimisasi seperti ini, yang dimana kita gunakan dapat memberitahukan nilai dari $r$, yang pastinnya tidak akan menskalakan permasalahan yang tidak dapat komputer klasik pecahkan. 

Tapi kegunaan dari latihan ini hanyalah memverifikasi bahwa algoritma Shor’s dapat berkerja sesuai harapan, dan kita tidak akan mengkhawatirkan fakta bahwa kita mencurangi untuk mendapatkan sirkuit untuk $U$.

<div id='u-definition'></div>
<div class="alert alert-block alert-success">

**Latihan 2a:** Membuat sebuah sirkuit ($U$) yang dapat memlakukan transformasi:

$$
\begin{aligned}
U|00\rangle &= |01\rangle \\
U|01\rangle &= |10\rangle \\
U|10\rangle &= |11\rangle \\
U|11\rangle &= |00\rangle \\
\end{aligned}
$$

dan jika di kontrol oleh qubit lainnya. Sirkuit tersebut akan berkerja pada 2 qubit target register yang bernama 'target', dan akan dikontrol oleh sebuah qubit register yang bernama 'kontrol'. Anda harus memberikan sirkuit yang anda selesaikan ke variabel '`cu`'.
    
</div>

In [ ]:
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, QuantumCircuit
c = QuantumRegister(1, 'control')
t = QuantumRegister(2, 'target')
cu = QuantumCircuit(c, t, name="Controlled 13^x mod 35")

# TULIS KODE ANDA DIANTARA BARIS INI - MULAI





# TULIS KODE ANDA DIANTARA BARIS INI - AKHIR

cu.draw('mpl')

Dan jalankan sel dibawah untuk mengecek jawaban anda:

In [ ]:
# Cek jawaban anda menggunakan kode dibawah
from qc_grader import grade_ex2a
grade_ex2a(cu)

Selamat! Anda telah menyelesaikan bagian tersulit. 

Kita membaca keluaran dari algoritma estimasi gelombang dengan mengukur qubit, jadi kita wajib memastikan register 'penghitung' cukup dapat untuk membaca $r$. Pada kasus kita, $r = 4$, yang dimana kita hanya memerlukan $\log_2(4) = 2$ qubit (menyontek lagi karena kita tahu $r$ sebelumnya), tapi sejak Santiago memiliki 5 qubit, dan kita hanya menggunakan 2 qubit sebagai register 'target', kita akan menggunakan sisa 3 qubit menjadi register penghitung.

Membuat estimasi gelombang pada $U$, kita perlu membuat sirkuit yang dapat melakukan $U^{2^x}$ ($U$ berulang $2^x$ kali) untuk setiap qubit (dengan index $x$) di dalam register $n$ qubit penghitung. Didalam kasus ini artinya kita perlu 3 sirkuit yang mengimplementasikan:

$$ U, \; U^2, \; \text{and} \; U^4 $$

Jadi tahap selanjutnya adalah membuat sirkuit yang dapat melakukan $U^2$ (yang sama dengan sirkuit yang mengaplikasikan $U$ secara dua kali).

<div class="alert alert-block alert-success">

**Latihan 2b:** Membuat sirkuit ($U^2$) yang melakukan transformasi:

$$
\begin{aligned}
U|00\rangle &= |10\rangle \\
U|01\rangle &= |11\rangle \\
U|10\rangle &= |00\rangle \\
U|11\rangle &= |01\rangle \\
\end{aligned}
$$

dan di kontrol oleh qubit lain. Sirkuit tersebut akan berkerja pada 2 qubit target register yang bernama 'target', dan akan dikontrol oleh sebuah qubit register yang bernama 'kontrol'. Anda harus memberikan sirkuit yang anda selesaikan ke variabel '`cu2`'.
</div>

In [ ]:
c = QuantumRegister(1, 'kontrol')
t = QuantumRegister(2, 'target')
cu2 = QuantumCircuit(c, t)

# TULIS KODE ANDA DIANTARA BARIS INI - MULAI





# TULIS KODE ANDA DIANTARA BARIS INI - AKHIR

cu2.draw('mpl')

Dan jalankan sel dibawah untuk mengecek jawaban anda:

In [ ]:
# Cek jawaban anda menggunakan code dibawah
from qc_grader import grade_ex2b
grade_ex2b(cu2)

Akhirnya, kita juga perlu sirkuit yang dapat mengaplikasikan $U$ 4 kali (kita memerlukan sirkuit $U^4$).  

<div class="alert alert-block alert-success">
    
**Latihan 2c:** Membuat sirkuit ($U^4$) yang dapat melakukan transformasi:

$$
\begin{aligned}
U|00\rangle &= |00\rangle \\
U|01\rangle &= |01\rangle \\
U|10\rangle &= |10\rangle \\
U|11\rangle &= |11\rangle \\
\end{aligned}
$$

dan di kontrol oleh qubit lain. Sirkuit tersebut akan berkerja pada 2 qubit target register yang bernama 'target', dan akan dikontrol oleh sebuah qubit register yang bernama 'kontrol'. Anda harus memberikan sirkuit yang anda selesaikan ke variabel '`cu4`'. _Petunjuk: Solusi terbaik adalah paling simpel._
</div>

In [ ]:
c = QuantumRegister(1, 'kontrol')
t = QuantumRegister(2, 'target')
cu4 = QuantumCircuit(c, t)

# TULIS CODE ANDA DIANTARA BARIS INI - MULAI





# TULIS KODE ANDA DIANTARA BARIS INI - AKHIR

cu4.draw('mpl')

Anda dapat mengecek jawaban anda mengunakan kode dibawah: 

In [ ]:
# Cek jawaban anda mengunakan kode berikut ini
from qc_grader import grade_ex2c
grade_ex2c(cu4)

<div class="alert alert-block alert-success">

**Latihan 2 final:** Sekarang kita harus mengkontrol $U$, $U^2$ dan $U^4$, kita dapat mengabungkan semua menjadi sirkuit yang menjalakn bagian kuantum dari algoritma Shor's.

Bagian initialisasi merupakan bagian mudah: kita perlu menaruh register penghitung ke keadaan $|{+}{+}{+}\rangle$ (yang kita dapat lakukan dengan menggunakan 3 gerbang-H) dan kita memerlukan register target menjadi keadaan $|1\rangle$ (yang dimana kita memetakan basis komputasioanal keadaan $|00\rangle$, sehingga kita tidak perlu membuat apapun disini). Kita akan lakuakan ini semua untuk anda.

Tugas _anda_ adalah membuat sirkuit yang menjalakan $U$-terkontrol, yang akan digunakan diantara initialisasi dan inverse transformasi Fourier kuantum . Secara formal, kita ingin sirkuit:


$$
CU_{c_0 t}CU^2_{c_1 t}CU^4_{c_2 t}
$$

Dimana $c_0$, $c_1$ dan $c_2$ adalah tiga qubit di register 'penghitung', $t$ adalah register ‘target’ , and $U$ adalah <a href="#u-definition">telah terdefinisi di bagian pertama latihan</a>. di dalam notasi ini, $CU_{a b}$ berarti $CU$ yang dikontrol oleh $a$ dan beraksi oleh $b$. Solusi termudah adalah mengabungkan semua sirkuit `cu`, `cu2` dan `cu4` yang anda buat, tapi anda akan mendapatkan sirkuit lebih efisien yang memiliki sifat yang sama!
    
</div>
<div class="alert alert-block alert-danger">
    
Sirkuit anda hanya boleh mengandung [CNOT](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) dan qubit tunggal [gerbang U](https://qiskit.org/documentation/stubs/qiskit.circuit.library.UGate.html). Nilai anda akan berasalah dari gerbang CNOT (semakin sedikit semakin bagus), karna gerbang multi qubit biasanya lebih sulit di jalankan di perangkat keras. Jika anda kesusahan untuk persyaratan yang ada, kami telah memasukan satu baris dekat pengumpulan yang akan mengkonversi sirkuit anda kedalam format yang ada, meskipun lebih disarankan anda lakukan menggunkan dengan tangan.
    
</div>

In [ ]:
# Kode untuk mengabungkan solusi sebelumnya ke pengumpulan terakhir anda
cqr = QuantumRegister(3, 'control')
tqr = QuantumRegister(2, 'target')
cux = QuantumCircuit(cqr, tqr)
solutions = [cu, cu2, cu4]
for i in range(3):
    cux = cux.compose(solutions[i], [cqr[i], tqr[0], tqr[1]])
cux.draw('mpl')

In [ ]:
# Cek jawaban anda dengan kode dibawah ini
from qc_grader import grade_ex2_final
# tutup komentar dibawah jika anda pingin mengkonversi sirkuit anda menjadi CNOT dan gerbang qubit tunggal.
#from qiskit import transpile
#cux = transpile(cux, basis_gates=['cx','u'])
grade_ex2_final(cux)

Jika anda sudah senang dengan sirkuit anda, anda dapat kumpul dibawah:

In [ ]:
# Kumpul jawaban anda. Anda dapat mengumpulkan kapan pun.
from qc_grader import submit_ex2_final
submit_ex2_final(cux)

Selamat! Anda telah menyelesaikan latihan ini. Lanjutkan membaca untuk melihat sirkuit anda digunakan untuk memfaktorkan angka 35, dan melihat bagaimana cara melakukannya.

## Mengunakan sirkuit anda untuk memfaktorkan 35

Sel kode dibawah akan mengambil submisi anda dari latihan dan menggunakannya untuk membuat sirkuit yag memberikan $\tfrac{s}{r}$, dimana $s$ adalah angka bilangan bulat acak diantara $0$ dan $r-1$, dan $r$ merupakan periode dari fungsi $f(x) = 13^x \bmod 35$.

In [ ]:
from qiskit.circuit.library import QFT
from qiskit import ClassicalRegister
# Membuat objek sikuit
cr = ClassicalRegister(3)
shor_circuit = QuantumCircuit(cqr, tqr, cr)

# Menginitialisasi qubit
shor_circuit.h(cqr)

# Tambah sirkuit anda
shor_circuit = shor_circuit.compose(cux)

# Lakukan inverse QFT dan mengekstrak keluarannya
shor_circuit.append(QFT(3, inverse=True), cqr)
shor_circuit.measure(cqr, cr)
shor_circuit.draw('mpl')

Mari kita trasnpile sirkuit dan lihat seberapa besar sirkuit tersebut, dan berapa banyak CNOT yang digunakan:

In [ ]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram
qasm_sim = Aer.get_backend('aer_simulator')
tqc = transpile(shor_circuit, basis_gates=['u', 'cx'], optimization_level=3)
print(f"kedalaman sirkuit: {tqc.depth()}")
print(f"sirkuit mengandung {tqc.count_ops()['cx']} CNOT")

Dan mari kita lihat apa yang kita dapat:

In [ ]:
counts = qasm_sim.run(tqc).result().get_counts()
plot_histogram(counts)

Asumsi semua berkerja dengan benar, kita seharusnya dapat melihat probabilitas yang sama dari pengukuran angka $0$, $2$, $4$ dan $8$. Ini disebabkan karena estimasi gelombang memberikan kita $2^n \cdot \tfrac{s}{r}$, dimana $n$ adalah jumlah qubit didalam register penghitung kita (disini $n = 3$, $s$ merupakan angka bulat acak $0$ dan $r-1$, dan $r$ merupakan angka yang kita ingin hitung). Mari kita konversi semua menjadi pecahan yang memberi tahu kita $s/r$ (yang dimana kita dapat hitung secara mudah):

In [ ]:
from fractions import Fraction
n = 3  #n merupakan jumlah dari register 'penghitung'
# Ulang setiap hasil pengukuran string
for measurement in counts.keys():
    # Konversi string binari ke 'int', dan bagi dengan 2^n
    decimal = int(measurement, 2)/2**n
    # Gunakan algoritma pecahan berkelanjutan untuk mengkonversi dari a/b
    print(Fraction(decimal).limit_denominator())

Kita dapat melihat penyebut dari beberapa hasil yang ada akan memberikan kita jawaban yang benar $r = 4$. kita dapat memverifikasi $r=4$ secara cepat:

In [ ]:
13**4 % 35

Jadi bagaimana kita dapat faktor dari ini? Dimana adanya probalitas tinggi yang dimana merupakan faktor persekutuan terbesar dari $N$ dan antara $a^{r/2}-1$ atau $a^{r/2}+1$ adalah faktor dari $N$, dan faktor persekutuan terbesar yang kita dapat hitung secara klasikal secara mudah.

In [ ]:
from math import gcd # faktor persekutuan terbesar
for x in [-1, 1]:
    print(f"faktor yangn ditebak: {gcd(13**(4//2)+x, 35)}")

Kita hanya perlu mencari satu faktor, dan dapat gunakan untuk membagi $N$ untuk mendapatkan faktor lainnya. Tetapi untuk kasusu ini, _kedua_ $a^{r/2}-1$ atau $a^{r/2}+1$ memberikan kita faktor $35$ f. Kita dapat memverifikasi lagi:

In [ ]:
7*5

## Jalakan di `ibmq_santiago`

Kami berjanji ini dapat berjalan di Santiago, jadi kami akan menunjukan bagaimana anda melakukannya. Didalam contoh ini kita akan menggunakan simulasi dari perangkat Santiago untuk kemudahan, tapi anda pdat mengantikan dengan perangkat asli jika anda mau:

In [ ]:
from qiskit.test.mock import FakeSantiago
from qiskit import assemble
from qiskit.visualization import plot_histogram
santiago = FakeSantiago()
real_device = False

## tutup komentar dibawah untuk menjalankan kode di perangkat asli
#from qiskit import IBMQ
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#santiago = provider.get_backend('ibmq_santiago')
#real_device = True

# Kita perlu transpile untuk Santiago
tqc = transpile(shor_circuit, santiago, optimization_level=3)

if not real_device:
    tqc = assemble(tqc)

# Jalankan sirkuit dan mencetak jumlah
counts = santiago.run(tqc).result().get_counts()
plot_histogram(counts)

Jika nilai anda cukup rendah, anda akan melihat kita mendapatkan probabilitas dari pengukuran $0$, $2$, $4$ atau $8$ seperti di simulasi. Anda akan melihat adanya hasil extra karena inakurasi dari prosesor dan interaksi asing pada qubit kita. `'noise'` ini akan semakin parah jika semakin panjang sirkuit yang kita miliki, semakin panjang waktu komputasi berarti semankin banyak potensi error yang ada. Karena itu kita wajib untuk curang untuk membuat sirkuit sekecil mungkin.

Pada masa depan, kuantum sistem kita akan cukup berkembang yang kita capat memulai menggunakan teknik mitigasi error lebih canggih untuk mengatasi permasalahan ini, yang dimana akan berarti kita dapat menjalankan sirkuit yang cukup besar untuk [mengerjakan algoritma Shor's tanpa mencurangi](https://arxiv.org/pdf/quant-ph/0205095.pdf).

## Additional information

**Created by:** Frank Harkins

**Indonesia Translate by:** Felix Montalfu

**Version:** 1.0.0